In [35]:
import yfinance as yf
import pandas as pd

def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    data = {
        "ticker": ticker,
        "price": info.get("currentPrice"),
        "pe": info.get("trailingPE"),
        "eps": info.get("trailingEps"),
        "marketCap": info.get("marketCap"),
        "pb": info.get("priceToBook"),
        "fiftyTwoWeekLow": info.get("fiftyTwoWeekLow"),
        "fiftyTwoWeekHigh": info.get("fiftyTwoWeekHigh")
    }
    return data

# Example tickers - Added more stocks
tickers = ["RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFC.NS", "BAJAJFINSV.NS"]

live_data = pd.DataFrame([get_stock_data(t) for t in tickers])
print(live_data)


          ticker   price         pe     eps     marketCap          pb  \
0    RELIANCE.NS  1518.2  24.742502   61.36  2.054500e+13    2.342467   
1         TCS.NS  3112.0  22.771841  136.66  1.125949e+13   10.580751   
2        INFY.NS  1505.8  21.502214   70.03  6.403998e+12  536.444640   
3        HDFC.NS     NaN        NaN     NaN           NaN         NaN   
4  BAJAJFINSV.NS  2064.9  34.466698   59.91  3.296090e+12    4.296504   

   fiftyTwoWeekLow  fiftyTwoWeekHigh  
0          1114.85           1551.00  
1          2866.60           4494.90  
2          1307.00           2006.45  
3              NaN               NaN  
4          1551.65           2195.00  


In [ ]:
http://localhost:5000

1. PE valuation

In [36]:
live_data["PE_Valuation"] = live_data["pe"].apply(
    lambda x: 1 if x < 15 else 0
)


2. Undervaluation via Price vs 52-week levels

In [37]:
live_data["Undervalued_52W"] = live_data.apply(
    lambda row: 1 if row["price"] < (row["fiftyTwoWeekLow"] * 1.2) else 0,
    axis=1
)


3. PEG Ratio (using TTM EPS Growth Proxy)

In [38]:
import pandas as pd

def get_screener_eps(stock_id):
    url = f"https://www.screener.in/company/{stock_id}/consolidated/"
    df = pd.read_html(url)
    # EPS table is usually df[2] or df[3]
    eps_table = df[2]  
    eps_values = eps_table.iloc[0].tolist()[1:]
    return eps_values

eps = get_screener_eps("RELIANCE")
print(eps)


['10%']


nse

In [39]:
import requests

url = "https://www.nseindia.com/api/quote-equity?symbol=RELIANCE"
headers = {"User-Agent": "Mozilla/5.0"}

data = requests.get(url, headers=headers).json()
# NSE API doesn't provide EPS, so we'll skip this and use yfinance/screener data
print("NSE data fetched (price info available)")


NSE data fetched (price info available)


NEXT STEP (Step 4): Combine All Data Into ONE

In [40]:
try:
    screener_eps = get_screener_eps("RELIANCE")  # list like [EPS2015, EPS2016, ...]
    live_data["EPS_hist"] = [screener_eps]
except:
    live_data["EPS_hist"] = None

try:
    # NSE API doesn't have EPS, so we skip it
    live_data["eps_nse"] = None
except:
    live_data["eps_nse"] = None


fill missing values

In [41]:
def final_eps(row):
    if row["eps_nse"] not in [0, None, ""] and pd.notna(row["eps_nse"]):
        return row["eps_nse"]
    if row["eps"] not in [0, None, ""] and pd.notna(row["eps"]):
        return row["eps"]
    if row["pe"] and row["price"]:
        return row["price"] / row["pe"]
    return None

live_data["final_EPS"] = live_data.apply(final_eps, axis=1)


Now Calculate PEG Ratio

In [42]:
# Calculate PEG Ratio first
live_data["PEG_Valuation"] = 0  # Initialize

# Only calculate if we have valid EPS and PE data
def calculate_peg(row):
    if row["final_EPS"] and row["final_EPS"] > 0 and row["pe"] and row["pe"] > 0:
        peg = row["pe"] / (row["final_EPS"] * 5)
        return 1 if peg < 1 else 0
    return 0

live_data["PEG_Valuation"] = live_data.apply(calculate_peg, axis=1)

live_data["G2G_Score"] = (
    live_data["PE_Valuation"] * 30 +
    live_data["PEG_Valuation"] * 30 +
    live_data["Undervalued_52W"] * 40
)


In [43]:
print(live_data[[
    "ticker", 
    "price", 
    "pe", 
    "final_EPS",
    "PE_Valuation", 
    "PEG_Valuation", 
    "Undervalued_52W",
    "G2G_Score"
]])


          ticker   price         pe  final_EPS  PE_Valuation  PEG_Valuation  \
0    RELIANCE.NS  1518.2  24.742502      61.36             0              1   
1         TCS.NS  3112.0  22.771841     136.66             0              1   
2        INFY.NS  1505.8  21.502214      70.03             0              1   
3        HDFC.NS     NaN        NaN        NaN             0              0   
4  BAJAJFINSV.NS  2064.9  34.466698      59.91             0              1   

   Undervalued_52W  G2G_Score  
0                0         30  
1                1         70  
2                1         70  
3                0          0  
4                0         30  


In [44]:
def g2g_model(ticker):

    # -------- yfinance ----------
    stock = yf.Ticker(ticker)
    info = stock.info
    
    price = info.get("currentPrice")
    pe    = info.get("trailingPE")
    eps_y = info.get("trailingEps")
    low52 = info.get("fiftyTwoWeekLow")
    pb    = info.get("priceToBook")

    # -------- screener ----------
    eps_screener = None
    try:
        eps_screener = get_screener_eps(ticker.split(".")[0])
    except:
        eps_screener = None

    # -------- NSE ----------
    # NSE API doesn't provide EPS, so skip it
    eps_nse = None
    
    # final EPS
    if eps_nse:
        eps_final = eps_nse
    elif eps_y:
        eps_final = eps_y
    else:
        eps_final = price / pe if pe and pe > 0 else None

    # Valuation flags
    pe_val = 1 if pe and pe < 15 else 0
    underval = 1 if price and low52 and price < low52 * 1.2 else 0
    peg = pe / (eps_final * 5) if eps_final and eps_final > 0 and pe and pe > 0 else None
    peg_val = 1 if peg and peg < 1 else 0

    g2g = (pe_val*30) + (peg_val*30) + (underval*40)

    return {
        "Ticker": ticker,
        "Price": price,
        "PE": pe,
        "EPS_Final": eps_final,
        "PEG": peg,
        "PE_Val": pe_val,
        "PEG_Val": peg_val,
        "52W_Underval": underval,
        "G2G_Score": g2g
    }


In [45]:
results = []
for ticker in tickers:
    result = g2g_model(ticker)
    results.append(result)

results_df = pd.DataFrame(results)
print(results_df.sort_values("G2G_Score", ascending=False))


          Ticker   Price         PE  EPS_Final       PEG  PE_Val  PEG_Val  \
1         TCS.NS  3112.0  22.771841     136.66  0.033326       0        1   
2        INFY.NS  1505.8  21.502214      70.03  0.061409       0        1   
0    RELIANCE.NS  1518.2  24.742502      61.36  0.080647       0        1   
4  BAJAJFINSV.NS  2064.9  34.466698      59.91  0.115062       0        1   
3        HDFC.NS     NaN        NaN        NaN       NaN       0        0   

   52W_Underval  G2G_Score  
1             1         70  
2             1         70  
0             0         30  
4             0         30  
3             0          0  


In [34]:
import requests
url = "https://www.nseindia.com/api/quote-equity?symbol=RELIANCE"
headers = {"User-Agent": "Mozilla/5.0"}
data = requests.get(url, headers=headers).json()
print(data.get("priceInfo", {}).keys())

dict_keys(['lastPrice', 'change', 'pChange', 'previousClose', 'open', 'close', 'vwap', 'stockIndClosePrice', 'lowerCP', 'upperCP', 'pPriceBand', 'basePrice', 'intraDayHighLow', 'weekHighLow', 'iNavValue', 'checkINAV', 'tickSize', 'ieq'])
